# HW06 - Trees

### Dataset

In this homework, we continue using the fuel efficiency dataset.
Download it from <a href='https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv'>here</a>.

You can do it with wget:

```bash
wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv
```

The goal of this homework is to create a regression model for predicting the car fuel efficiency (column `'fuel_efficiency_mpg'`).



### Preparing the dataset 

Preparation:

* Fill missing values with zeros.
* Do train/validation/test split with 60%/20%/20% distribution. 
* Use the `train_test_split` function and set the `random_state` parameter to 1.
* Use `DictVectorizer(sparse=True)` to turn the dataframes into matrices.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.tree import export_text
from IPython.display import display
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb


In [2]:
df = pd.read_csv(
    "https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv"
)

In [3]:
# The target variable is 'fuel_efficiency_mpg'.
target = "fuel_efficiency_mpg"

In [4]:
df.columns = df.columns.str.lower().str.replace(" ", "_")

string_columns = list(df.dtypes[df.dtypes == "object"].index)

for col in string_columns:
    df[col] = df[col].str.lower().str.replace(" ", "_")

In [5]:
df.head()

,engine_displacement,num_cylinders,horsepower,vehicle_weight,acceleration,model_year,origin,fuel_type,drivetrain,num_doors,fuel_efficiency_mpg
0,170,3.0,159.0,3413.433759,17.7,2003,europe,gasoline,all-wheel_drive,0.0,13.231729
1,130,5.0,97.0,3149.664934,17.8,2007,usa,gasoline,front-wheel_drive,0.0,13.688217
2,170,NaN,78.0,3079.038997,15.1,2018,europe,gasoline,front-wheel_drive,0.0,14.246341
3,220,4.0,NaN,2542.392402,20.2,2009,usa,diesel,all-wheel_drive,2.0,16.912736
4,210,1.0,140.0,3460.870990,14.4,2009,europe,gasoline,all-wheel_drive,2.0,12.488369


In [6]:
# Separate the target variable before any splitting/filling
y = df[target]
df = df.drop(columns=[target])
# Fill missing values
df = df.fillna(0)

In [7]:
# Perform Train/Validation/Test Split (60%/20%/20%)
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

# Separate target variables (y) for each set
y_train = y.loc[df_train.index]
y_val = y.loc[df_val.index]
y_test = y.loc[df_test.index]

# Reset indices for clean conversion to dicts
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = y_train.reset_index(drop=True)
y_val = y_val.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

# Convert DataFrames to Dicts (Required for DictVectorizer)
train_dicts = df_train.to_dict(orient="records")
val_dicts = df_val.to_dict(orient="records")
test_dicts = df_test.to_dict(orient="records")

# Use DictVectorizer to turn dicts into feature matrices
dv = DictVectorizer(sparse=True)
X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)
X_test = dv.transform(test_dicts)

In [8]:
def rmse(y_true, y_pred):
    """Calculates Root Mean Squared Error (RMSE)"""
    return np.sqrt(mean_squared_error(y_true, y_pred))

## Question 1

Let's train a decision tree regressor to predict the `fuel_efficiency_mpg` variable. 

* Train a model with `max_depth=1`.


Which feature is used for splitting the data?


* `'vehicle_weight'`
* `'model_year'`
* `'origin'`
* `'fuel_type'`

In [9]:
# Train a Decision Tree Regressor with max_depth=1
dt = DecisionTreeRegressor(max_depth=1, random_state=1)
dt.fit(X_train, y_train)

# Get the feature importance
feature_importances = pd.Series(
    dt.feature_importances_, index=dv.get_feature_names_out()
)
most_important_feature = feature_importances.idxmax()
split_value = feature_importances.max()

print(f"Q1 - Feature used for splitting (max_depth=1): {most_important_feature}")

Q1 - Feature used for splitting (max_depth=1): vehicle_weight


## Question 2

Train a random forest regressor with these parameters:

* `n_estimators=10`
* `random_state=1`
* `n_jobs=-1` (optional - to make training faster)


What's the RMSE of this model on the validation data?

* 0.045
* 0.45
* 4.5
* 45.0

In [10]:
# Train a Random Forest Regressor with n_estimators=10
rf = RandomForestRegressor(n_estimators=10, random_state=1, n_jobs=-1)
rf.fit(X_train, y_train)

# Predict on validation data
y_pred_val = rf.predict(X_val)

# Calculate RMSE on validation data
rmse_val_q2 = rmse(y_val, y_pred_val)

print(f"Q2 - RMSE on validation data (n_estimators=10): {rmse_val_q2:.3f}")

Q2 - RMSE on validation data (n_estimators=10): 0.460


## Question 3

Now let's experiment with the `n_estimators` parameter

* Try different values of this parameter from 10 to 200 with step 10.
* Set `random_state` to `1`.
* Evaluate the model on the validation dataset.


After which value of `n_estimators` does RMSE stop improving?
Consider 3 decimal places for calculating the answer.

- 10
- 25
- 80
- 200

If it doesn't stop improving, use the latest iteration number in
your answer.

In [11]:
# Experiment with n_estimators from 10 to 200 with step 10
scores = []
n_estimators_list = range(10, 201, 10)

for n in n_estimators_list:
    rf = RandomForestRegressor(n_estimators=n, random_state=1, n_jobs=-1)
    rf.fit(X_train, y_train)

    y_pred_val = rf.predict(X_val)
    rmse_val = rmse(y_val, y_pred_val)

    scores.append((n, rmse_val))

df_scores_q3 = pd.DataFrame(scores, columns=["n_estimators", "rmse"])

# Find when RMSE stops improving (consider 3 decimal places)
# We look for the point where the difference between two consecutive RMSEs is very small (e.g., < 0.001) or starts increasing.
best_n = df_scores_q3.iloc[0]["n_estimators"]
min_rmse = df_scores_q3.iloc[0]["rmse"]

print("Q3 - n_estimators vs RMSE:")
print(df_scores_q3.to_string(index=False))

# Check when the improvement is marginal (e.g., less than 0.001)
improvement_threshold = 0.001
n_stop_improving = None

for i in range(1, len(df_scores_q3)):
    prev_rmse = df_scores_q3.iloc[i - 1]["rmse"]
    current_rmse = df_scores_q3.iloc[i]["rmse"]

    if (prev_rmse - current_rmse) < improvement_threshold:
        n_stop_improving = df_scores_q3.iloc[i]["n_estimators"]
        break

print(f"\nQ3 - RMSE stops improving after n_estimators={n_stop_improving}")

Q3 - n_estimators vs RMSE:
 n_estimators     rmse
           10 0.459578
           20 0.453591
           30 0.451687
           40 0.448721
           50 0.446657
           60 0.445460
           70 0.445126
           80 0.444984
           90 0.444861
          100 0.444652
          110 0.443579
          120 0.443912
          130 0.443703
          140 0.443355
          150 0.442898
          160 0.442761
          170 0.442801
          180 0.442362
          190 0.442494
          200 0.442479

Q3 - RMSE stops improving after n_estimators=70.0


## Question 4

Let's select the best `max_depth`:

* Try different values of `max_depth`: `[10, 15, 20, 25]`
* For each of these values,
  * try different values of `n_estimators` from 10 till 200 (with step 10)
  * calculate the mean RMSE 
* Fix the random seed: `random_state=1`


What's the best `max_depth`, using the mean RMSE?

* 10
* 15
* 20
* 25

In [12]:
# Select the best max_depth
max_depths = [10, 15, 20, 25]
n_estimators_list = range(10, 201, 10)
scores_q4 = []

for depth in max_depths:
    rmse_per_depth = []
    for n in n_estimators_list:
        rf = RandomForestRegressor(
            n_estimators=n, max_depth=depth, random_state=1, n_jobs=-1
        )
        rf.fit(X_train, y_train)

        y_pred_val = rf.predict(X_val)
        rmse_val = rmse(y_val, y_pred_val)
        rmse_per_depth.append(rmse_val)

    # Calculate mean RMSE for the current max_depth
    mean_rmse = np.mean(rmse_per_depth)
    scores_q4.append((depth, mean_rmse))

df_scores_q4 = pd.DataFrame(scores_q4, columns=["max_depth", "mean_rmse"])
best_max_depth = df_scores_q4.loc[df_scores_q4["mean_rmse"].idxmin()]

print("Q4 - max_depth vs Mean RMSE:")
print(df_scores_q4.to_string(index=False))

print(
    f"\nQ4 - Best max_depth based on mean RMSE: {best_max_depth['max_depth']} (Mean RMSE: {best_max_depth['mean_rmse']:.4f})"
)

Q4 - max_depth vs Mean RMSE:
 max_depth  mean_rmse
        10   0.441808
        15   0.445417
        20   0.446253
        25   0.445910

Q4 - Best max_depth based on mean RMSE: 10.0 (Mean RMSE: 0.4418)


# Question 5

We can extract feature importance information from tree-based models. 

At each step of the decision tree learning algorithm, it finds the best split. 
When doing it, we can calculate "gain" - the reduction in impurity before and after the split. 
This gain is quite useful in understanding what are the important features for tree-based models.

In Scikit-Learn, tree-based models contain this information in the
[`feature_importances_`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor.feature_importances_)
field. 

For this homework question, we'll find the most important feature:

* Train the model with these parameters:
  * `n_estimators=10`,
  * `max_depth=20`,
  * `random_state=1`,
  * `n_jobs=-1` (optional)
* Get the feature importance information from this model


What's the most important feature (among these 4)? 

* `vehicle_weight`
*	`horsepower`
* `acceleration`
* `engine_displacement`	

In [13]:
# Train Random Forest Regressor to get feature importance
rf_q5 = RandomForestRegressor(n_estimators=10, max_depth=20, random_state=1, n_jobs=-1)
rf_q5.fit(X_train, y_train)

# Get feature importance
feature_importances_q5 = pd.Series(
    rf_q5.feature_importances_, index=dv.get_feature_names_out()
)

# Filter for the 4 features in question
target_features = [
    "vehicle_weight",
    "horsepower",
    "acceleration",
    "engine_displacement",
]
filtered_importance = feature_importances_q5[target_features]
most_important_feature_q5 = filtered_importance.idxmax()

print(
    f"Q5 - Feature importances (filtered):\n{filtered_importance.sort_values(ascending=False)}"
)
print(f"\nQ5 - Most important feature: {most_important_feature_q5}")

Q5 - Feature importances (filtered):
vehicle_weight         0.959150
horsepower             0.015998
acceleration           0.011480
engine_displacement    0.003273
dtype: float64

Q5 - Most important feature: vehicle_weight


## Question 6

Now let's train an XGBoost model! For this question, we'll tune the `eta` parameter:

* Install XGBoost
* Create DMatrix for train and validation
* Create a watchlist
* Train a model with these parameters for 100 rounds:

```
xgb_params = {
    'eta': 0.3, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}
```

Now change `eta` from `0.3` to `0.1`.

Which eta leads to the best RMSE score on the validation dataset?

* 0.3
* 0.1
* Both give equal value

In [16]:
# 1. Create DMatrix for train and validation
# FIX: Get feature names and convert them to a Python list
feature_names = dv.get_feature_names_out().tolist()

# Use the 'feature_names' list when creating DMatrix
dtrain = xgb.DMatrix(X_train, label=y_train, feature_names=feature_names)
dval = xgb.DMatrix(X_val, label=y_val, feature_names=feature_names)

# 2. Define watchlist
watchlist = [(dtrain, "train"), (dval, "val")]

xgb_params = {
    "max_depth": 6,
    "min_child_weight": 1,
    "objective": "reg:squarederror",  # Regression objective
    "nthread": 8,
    "seed": 1,
    "verbosity": 0,  # Set to 0 to suppress excessive output during training
}
num_round = 100


# Function to train and evaluate XGBoost
def train_and_eval_xgb(eta_value, xgb_params, num_round, watchlist):
    params = xgb_params.copy()
    params["eta"] = eta_value

    # Train the model
    model = xgb.train(
        params, dtrain, num_round, watchlist, evals_result={}, verbose_eval=False
    )

    # Predict on validation set
    y_pred_val_xgb = model.predict(dval)
    # Calculate RMSE
    rmse_val_xgb = rmse(y_val, y_pred_val_xgb)

    return rmse_val_xgb


# Case 1: eta = 0.3
rmse_03 = train_and_eval_xgb(0.3, xgb_params, num_round, watchlist)

# Case 2: eta = 0.1
rmse_01 = train_and_eval_xgb(0.1, xgb_params, num_round, watchlist)

print(f"Q6 - RMSE (eta=0.3): {rmse_03:.4f}")
print(f"Q6 - RMSE (eta=0.1): {rmse_01:.4f}")

# Determine the best eta
if rmse_03 < rmse_01:
    best_eta = 0.3
elif rmse_01 < rmse_03:
    best_eta = 0.1
else:
    best_eta = "Both equal"

print(f"\nQ6 - The best eta is: {best_eta}")

d:\001_Weiterbildung\01_Programming\Zoomcamp\machine-learning-zoomcamp\.venv\Lib\site-packages\xgboost\core.py:771: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)
d:\001_Weiterbildung\01_Programming\Zoomcamp\machine-learning-zoomcamp\.venv\Lib\site-packages\xgboost\core.py:771: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)


Q6 - RMSE (eta=0.3): 0.4502
Q6 - RMSE (eta=0.1): 0.4262

Q6 - The best eta is: 0.1
